# Atmospheric deposition dataset extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is used to retrieve and concatenate the atmospheric deposition data into a table for publication alongisde the used data.


## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas=2.1.3
* scipy=1.9.0
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* 


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* 
## Observations
* 

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import tqdm as tqdm
import os
import glob
import warnings
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib as mpl
import glob
from matplotlib.lines import Line2D
import matplotlib as mpl
import rasterio
from collections import defaultdict


# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = ".."
# Suppress all warnings
warnings.filterwarnings("ignore")

* #### The users should NOT change anything in the code below here. 

In [3]:
# Non-editable variables:
PATH_OUTPUT = r"results\catchment_aggregated_data\atmospheric_deposition"

# Set the directory:
os.chdir(PATH)

In [4]:
os.getcwd()

'c:\\Users\\nascimth\\OneDrive - Eawag\\Eawag\\Papers\\CAMELS_CH_Chem\\GitHub\\CAMELS_CH_chem'

# Import data
* Full table

In [5]:
path_data = r"C:\Users\nascimth\Documents\data\CAMELS_CH_Chem\data"

In [8]:
catchments = gpd.read_file("results\catchment_delineations\camels_ch_del\camels_ch_chem_catchment_boundaries.shp")

catchments

,gauge_id,sensor_id,nawaf_id,nawat_id,isot_id,gauge_name,water_body,gauge_east,gauge_nort,gauge_lon,gauge_lat,area,geometry
0,2009,2009.0,1837.0,1837.0,NIO04,Porte du Scex,Rhône,557660,133280,6.89,46.35,5239.4,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
1,2011,2011.0,NaN,4070.0,None,Sion,Rhône,593770,118630,7.36,46.22,3372.4,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
2,2016,2016.0,1833.0,1833.0,NIO02,Brugg,Aare,657000,259360,8.19,47.48,11681.3,"POLYGON Z ((2655969.680 1259695.589 0.000, 265..."
3,2018,2018.0,1835.0,1339.0,None,Mellingen,Reuss,662830,252580,8.27,47.42,3385.8,"POLYGON Z ((2663723.380 1252919.068 0.000, 266..."
4,2019,2019.0,NaN,1852.0,NIO01,Brienzwiler,Aare,649930,177380,8.09,46.75,555.2,"POLYGON Z ((2669196.412 1183579.510 0.000, 266..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2617,2617.0,NaN,NaN,None,Müstair,Rom,830800,168700,10.45,46.63,128.6,"POLYGON Z ((2820942.826 1171469.984 0.000, 282..."
112,2623,2623.0,NaN,NaN,None,Oberwald,Rhone,669900,154075,8.35,46.53,93.3,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
113,2634,2634.0,6169.0,1181.0,None,Emmen,Kleine Emme,663700,213630,8.28,47.07,478.3,"POLYGON Z ((2653429.237 1216261.807 0.000, 265..."
114,2635,2635.0,NaN,NaN,None,"Einsiedeln, Gross",Grossbach,700710,218125,8.77,47.11,8.9,"POLYGON Z ((2701144.527 1218073.633 0.000, 270..."


In [17]:
# List of filenames
filenames_rasters = [
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2020.tif'
]

# Group files by their prefixes
file_groups = defaultdict(list)

for filename in filenames_rasters:
    prefix = filename.split("/")[-1].split("_")[0]  # Extract prefix (e.g., 'dhno3gas')
    file_groups[prefix].append(filename)

In [18]:
file_groups.keys()

dict_keys(['dhno3gas', 'dnh3gas', 'dnh4total', 'dno2gas', 'dno3total', 'dntotal'])

## Reproject to projected coordinates system


In [11]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:2056'

# Reproject the GeoDataFrame to the target CRS
catchments_unique_reprojected = catchments.to_crs(target_crs)

In [12]:
# Set the index and adjust to int (instead of float)
catchments_unique_reprojected.set_index("gauge_id", inplace=True)
catchments_unique_reprojected.index = catchments_unique_reprojected.index.astype(int)

## Computation processes


In [13]:
# Convert DataFrame to GeoDataFrame with valid geometries
catchments_unique_reprojected = gpd.GeoDataFrame(
    catchments_unique_reprojected,
    geometry=catchments_unique_reprojected['geometry'],  # Use the existing geometry column
    crs="EPSG:2056"  # Adjust CRS as necessary
)

In [31]:
from tqdm import tqdm
import pandas as pd
import rasterio
import numpy as np
from rasterio.mask import geometry_mask
from shapely.geometry import mapping
from rasterio.features import geometry_mask

# Define prefixes for their names based on the order of lecture
prefix_values = ["1990_", "2000_", "2005_", "2010_", "2015_", "2020_"]

compounds = ['dhno3gas', 'dnh3gas', 'dnh4total', 'dno2gas', 'dno3total', 'dntotal']

# Iterate over each unique code
for code in tqdm(catchments_unique_reprojected.index):
    # Initialize an empty DataFrame to store the results for the current code
    code_df = pd.DataFrame()

    for compound in compounds:
        # Initialize a DataFrame for the current compound
        compound_data = []
        compound_df = pd.DataFrame()

        i = 0
        filenames = file_groups[compound]

        for filename in filenames:
            # Create lists to store the results
            avg_values = []

            # Load your raster file
            with rasterio.open(path_data+"\\"+ filename) as src:
                geom = catchments_unique_reprojected.loc[[code]]

                # Check if the geometry is empty or invalid
                geometry = geom['geometry'].iloc[0]

                if geometry is None or geometry.is_empty or not geometry.is_valid:
                    avg_value = np.nan
                else:
                    
                    # Convert geometry to GeoJSON-like dict for rasterio
                    geom_mapping = [mapping(geometry)]
                    
                    # Create a mask for the geometry
                    mask = geometry_mask(geom_mapping, out_shape=src.shape, transform=src.transform, invert=True)

                    # Read the values within the geometry from the raster
                    values = src.read(1, masked=True)
                    values = values[mask]

                # Calculate statistics only if there are valid values in the 'values' array
                if len(values) > 0:
                    avg_value = np.sum(values)
                else:
                    # Handle the case when there are no valid values
                    avg_value = np.nan

                # Append the result for the current year (prefix)
                compound_data.append(avg_value)

            i += 1

        # Add the compound data as a column to the code DataFrame
        code_df[compound] = compound_data

    # Add year prefixes as the index
    code_df.index = [prefix[:-1] for prefix in prefix_values]

    
    code_df.index = code_df.index.astype(int)

    # Generate a full range of years from 1980 to 2019
    full_range = pd.DataFrame(index=range(1990, 2021))

    # Reindex the dataframe to include all years
    code_df_interpolated = code_df.reindex(full_range.index)

    # Interpolate missing values
    code_df_interpolated = code_df_interpolated.interpolate(method='linear')


    code_df_interpolated = code_df_interpolated.round(4)
    code_df_interpolated.index.name = "date"

    code_df_interpolated.to_csv(PATH_OUTPUT + "/CAMELS_CH_Chem_atm_"+str(code)+".csv", encoding='latin')

100%|██████████| 116/116 [05:56<00:00,  3.07s/it]


# End